In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
%cd /content/gdrive/My Drive/Kaggle
!kaggle competitions download -c instacart-market-basket-analysis
!unzip products.csv.zip -d '/content/'
!unzip orders.csv.zip -d '/content/'
!unzip order_products__train.csv.zip -d '/content/'
!unzip order_products__prior.csv.zip -d '/content/'
!unzip aisles.csv.zip -d '/content/'
!unzip departments.csv.zip -d '/content/'

/content/gdrive/My Drive/Kaggle
departments.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
products.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
order_products__train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
order_products__prior.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
aisles.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  products.csv.zip
  inflating: /content/products.csv   
   creating: /content/__MACOSX/
  inflating: /content/__MACOSX/._products.csv  
Archive:  orders.csv.zip
  inflating: /content/orders.csv     
  inflating: /content/__MA

In [ ]:
#import of liberys
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

In [ ]:
#reading all csv files.
%cd "/content"
data_path="/content/"
prior_data = pd.read_csv( data_path + 'order_products__prior.csv')
train_data = pd.read_csv( data_path + 'order_products__train.csv')
orders = pd.read_csv( data_path + 'orders.csv')
products=pd.read_csv( data_path + 'products.csv')
aisles=pd.read_csv( data_path + 'aisles.csv')
departments=pd.read_csv( data_path + 'departments.csv')

/content


In [ ]:
#replace nan value in order.csv
avg_day_since_prior=orders[orders['days_since_prior_order'].apply(str)!='NaN']
avg_day_since_prior=(dict(avg_day_since_prior.groupby('user_id')['days_since_prior_order'].agg(np.median)))
for i in range(len(orders)):
  if str(orders['days_since_prior_order'][i])=='nan':
    orders['days_since_prior_order'].values[i]=avg_day_since_prior[orders['user_id'].values[i]]    

Here I pose this problem as binary classfication problem so,we are creating a data frame consist of<br>
    1. user_id
    2. product_id(belongs to ordered products by the user_id in history)
    3. reordered(will the given product_id is reordered by the given user_id in next order)

In [ ]:
#first we create a data frame in whice each row belongs to user_id ,product_id(all product borought in past by that user),some feature extracted from prior_dataset

In [ ]:
#first we create a data frame in whice each row belongs to user_id ,product_id(all product borought in past by that user),some feature extracted from prior_dataset
"""here we are pose this problem as binary classfication problem
    so,we are creating a data frame consist of
    1. user_id
    2. product_id(belongs to ordered products by the user_id in history)"""
users=pd.DataFrame(set(orders[orders['eval_set']=='prior']['user_id']),columns=['user_id'])
user_record=pd.merge(orders,users,on='user_id',how='inner')
order_product_prior=pd.merge(user_record,prior_data,on='order_id',how='inner')[['user_id','product_id']]
final_df=order_product_prior.drop_duplicates(subset=['user_id', 'product_id'], keep='first')
final_df.head()

,user_id,product_id
0,1,196
1,1,14084
2,1,12427
3,1,26088
4,1,26405


Now we will create some feature such that<br>
1.some feature releated to user<br>
2.some feature releated to item<br>
3.some feature releated to user and item.

<h3>user and item releated feature:-</h3><br>
 After research i have found some features that may help us such features are:-<br>

1.   nb_times_userA_order_itemB
2.   avg_pos_cart_userA_itemB
3.   nb_order_userA_itemB_last5
4.   median_day_since_prior_order_userA_itemB
5.   streak_userA_itemB
6.   mean_order_diff_userA_itemB
7.   nb_order_userA_not_ordered_itemB
8.   difference_mean_order_diff_nb_order_not_ordered_userA_itemB
9.   order_ratio_userA_itemB
10.   no_order_department_userA_itemB
11.   avg_add_to_cart_order_department_userA_itemB
12.  no_order_aisle_userA_itemB
13.  avg_add_to_cart_order_aisle_userA_itemB










In [ ]:
#nb_times_userA_order_itemB feature is calculated
user_item=pd.merge(prior_data,orders,on='order_id',how='inner')
temp_dict=dict(user_item.groupby(['user_id','product_id'])['user_id'].count())
nb_times_userA_order_itemB=[]
for idx,row in tqdm(final_df.iterrows()):
  temp=temp_dict[(row['user_id'],row['product_id'])]
  nb_times_userA_order_itemB.append(temp)
final_df['nb_times_userA_order_itemB']=nb_times_userA_order_itemB

13307953it [15:28, 14326.62it/s]


In [ ]:
#avg_pos_cart_userA_itemB feature is calculated
user_item=pd.merge(prior_data,orders,on='order_id',how='inner')
temp_dict=dict(user_item.groupby(['user_id','product_id'])['add_to_cart_order'].mean())
avg_pos_cart_userA_itemB=[]
for idx,row in tqdm(final_df.iterrows()):
  temp=temp_dict[(row['user_id'],row['product_id'])]
  avg_pos_cart_userA_itemB.append(temp)
final_df['avg_pos_cart_userA_itemB']=avg_pos_cart_userA_itemB

13307953it [15:17, 14500.99it/s]


In [ ]:
#nb_order_userA_itemB_last5 feature is calculated
recent_order=(orders[orders['eval_set']=='prior']).groupby('user_id').tail(5)
recent_user_item=pd.merge(prior_data,recent_order,on='order_id',how='inner')
temp_dict=dict(recent_user_item.groupby(['user_id','product_id'])['user_id'].count())
nb_order_userA_itemB_last5=[]
for idx,row in tqdm(final_df.iterrows()):
  try:
    temp=temp_dict[(row['user_id'],row['product_id'])]
    nb_order_userA_itemB_last5.append(temp)
  except KeyError:
    nb_order_userA_itemB_last5.append(0)
final_df['nb_order_userA_itemB_last5']=nb_order_userA_itemB_last5

13307953it [12:30, 17742.35it/s]


In [ ]:
#median_day_since_prior_order_userA_itemB feature is calculated
user_item=pd.merge(prior_data,orders,on='order_id',how='inner')
temp_dict=dict(user_item.groupby(['user_id','product_id'])['days_since_prior_order'].median())
median_day_since_prior_order_userA_itemB=[]
for idx,row in tqdm(final_df.iterrows()):
  temp=temp_dict[(row['user_id'],row['product_id'])]
  median_day_since_prior_order_userA_itemB.append(temp)
final_df['median_day_since_prior_order_userA_itemB']=median_day_since_prior_order_userA_itemB

13307953it [12:42, 17453.03it/s]


In [ ]:
# Python3 program to find largest consecutive 
# numbers present in arr. 
#https://www.geeksforgeeks.org/maximum-consecutive-numbers-present-array/
def find_max_streak(arr): 
	'''We insert all the array elements into unordered set.'''

	S = set(arr); 
	n=len(arr)
	# check each possible sequence from the start 
	# then update optimal length 
	ans = 0; 
	for i in range(n): 
		
		# if current element is the starting 
		# element of a sequence 
		if S.__contains__(arr[i]): 
			
			# Then check for next elements in the 
			# sequence 
			j = arr[i]; 
			
			# increment the value of array element 
			# and repeat search in the set 
			while(S.__contains__(j)): 
				j += 1; 

			# Update optimal length if this length 
			# is more. To get the length as it is 
			# incremented one by one 
			ans = max(ans, j - arr[i]); 
	return ans; 


In [ ]:
def mean_order_diff(arr):
  n=len(arr)
  arr=sorted(arr)
  temp=[]
  for i in range(n-1):
    temp.append(arr[i+1]-arr[i])
  if len(temp)==0:
      return 0.0
  return np.mean(temp)

In [ ]:
#streak_userA_itemB feature is calculated
user_item=pd.merge(prior_data,orders,on='order_id',how='inner')
temp=dict(user_item.groupby(['user_id','product_id'])['order_number'].apply(list))
streak_userA_itemB=[]
for idx,row in tqdm(final_df.iterrows()):
  temp_arr=temp[row['user_id'],row['product_id']]
  streak_userA_itemB.append(find_max_streak(temp_arr))
final_df['streak_userA_itemB']=streak_userA_itemB

13307953it [13:13, 16761.55it/s]


In [ ]:
#mean_order_diff_userA_itemB feature is calculated
user_item=pd.merge(prior_data,orders,on='order_id',how='inner')[['user_id','product_id','order_number']]
temp=dict(user_item.groupby(['user_id','product_id'])['order_number'].apply(list))
mean_order_diff_userA_itemB=[]
for idx,row in tqdm(final_df.iterrows()):
  temp_arr=temp[row['user_id'],row['product_id']]
  mean_order_diff_userA_itemB.append(mean_order_diff(temp_arr))
final_df['mean_order_diff_userA_itemB']=mean_order_diff_userA_itemB

13307953it [15:20, 14460.87it/s]


In [ ]:
#nb_order_userA_not_ordered_itemB feature is calculated
train_order=orders[orders['eval_set']=='prior']
user_item=pd.merge(prior_data,train_order,on='order_id',how='inner')[['user_id','order_number']]
last_order_user=dict(user_item.groupby(['user_id'])['order_number'].max())
user_item=pd.merge(prior_data,train_order,on='order_id',how='inner')[['user_id','product_id','order_number']]
last_order_user_item=dict(user_item.groupby(['user_id','product_id'])['order_number'].max())
nb_order_userA_not_ordered_itemB=[]
for idx,row in tqdm(final_df.iterrows()):
  temp1=last_order_user_item[row['user_id'],row['product_id']]
  temp2=last_order_user[row['user_id']]
  nb_order_userA_not_ordered_itemB.append(temp2-temp1)
final_df['nb_order_userA_not_ordered_itemB']=nb_order_userA_not_ordered_itemB

13307953it [16:16, 13625.65it/s]


In [ ]:
#difference_mean_order_diff_nb_order_not_ordered_userA_itemB feature is calculated
def cal_sub_col(row):
    return row['mean_order_diff_userA_itemB'] -row['nb_order_userA_not_ordered_itemB']
final_df['difference_mean_order_diff_nb_order_not_ordered_userA_itemB'] = final_df.apply(cal_sub_col, axis=1)

In [ ]:
#order_ratio_userA_itemB feature is calculated
train_order=orders[orders['eval_set']=='prior']
user_item=pd.merge(prior_data,train_order,on='order_id',how='inner')[['user_id','order_number']]
last_order_user=dict(user_item.groupby(['user_id'])['order_number'].max())
user_item=pd.merge(prior_data,train_order,on='order_id',how='inner')[['user_id','product_id','order_number']]
last_order_user_item=dict(user_item.groupby(['user_id','product_id'])['order_number'].max())
def order_ratio(row):
  return row['nb_times_userA_order_itemB']/last_order_user[row['user_id']]
final_df['order_ratio_userA_itemB'] = final_df.apply(order_ratio, axis=1)

In [ ]:
#no_order_aisle_userA_itemB and avg_add_to_cart_order_aisle_userA_itemB feature is calculated
data=orders[orders['eval_set']=='prior']
data=pd.merge(data,prior_data,on='order_id',how='inner')
data=pd.merge(data,products,on='product_id',how='inner')
data=pd.merge(data,aisles,on='aisle_id',how='inner')
final_df=pd.merge(final_df,products[['product_id','aisle_id']],on='product_id',how='inner')

no_order_aisle_userA_itemB=data.groupby(['user_id','aisle_id'])['product_id'].count().reset_index(name='no_order_aisle_userA_itemB')
final_df=pd.merge(final_df,no_order_aisle_userA_itemB,on=['user_id','aisle_id'],how='inner')
avg_add_to_cart_order_aisle_userA_itemB=data.groupby(['user_id','aisle_id'])['add_to_cart_order'].mean().reset_index(name='avg_add_to_cart_order_aisle_userA_itemB')
final_df=pd.merge(final_df,avg_add_to_cart_order_aisle_userA_itemB,on=['user_id','aisle_id'],how='inner')
final_df=final_df.drop(['aisle_id'], axis=1)

In [ ]:
#no_order_department_userA_itemB and avg_add_to_cart_order_department_userA_itemB feature is calculated
data=orders[orders['eval_set']=='prior']
data=pd.merge(data,prior_data,on='order_id',how='inner')
data=pd.merge(data,products,on='product_id',how='inner')
data=pd.merge(data,departments,on='department_id',how='inner')
final_df=pd.merge(final_df,products[['product_id','department_id']],on='product_id',how='inner')
no_order_department_userA_itemB=data.groupby(['user_id','department_id'])['product_id'].count().reset_index(name='no_order_department_userA_itemB')
final_df=pd.merge(final_df,no_order_department_userA_itemB,on=['user_id','department_id'],how='inner')
avg_add_to_cart_order_department_userA_itemB=data.groupby(['user_id','department_id'])['add_to_cart_order'].mean().reset_index(name='avg_add_to_cart_order_department_userA_itemB')
final_df=pd.merge(final_df,avg_add_to_cart_order_department_userA_itemB,on=['user_id','department_id'],how='inner')
final_df=final_df.drop(['department_id'], axis=1)

In [ ]:
final_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/final_df.csv', index=False,header=True)

In [ ]:
final_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/final_df.csv')

In [ ]:
final_df.head()

,user_id,product_id,nb_times_userA_order_itemB,avg_pos_cart_userA_itemB,nb_order_userA_itemB_last5,median_day_since_prior_order_userA_itemB,streak_userA_itemB,mean_order_diff_userA_itemB,nb_order_userA_not_ordered_itemB,difference_mean_order_diff_nb_order_not_ordered_userA_itemB,order_ratio_userA_itemB,no_order_aisle_userA_itemB,avg_add_to_cart_order_aisle_userA_itemB,no_order_department_userA_itemB,avg_add_to_cart_order_department_userA_itemB
0,1,196,10,1.400000,5,19.5,10,1.00,0,1.00,1.000000,13,1.769231,13,1.769231
1,1,46149,3,3.000000,3,14.0,3,1.00,0,1.00,0.300000,13,1.769231,13,1.769231
2,15,196,5,2.200000,3,7.0,2,1.75,0,1.75,0.227273,9,2.333333,20,2.400000
3,15,30292,1,3.000000,0,17.0,1,0.00,17,-17.00,0.045455,9,2.333333,20,2.400000
4,15,48142,3,2.333333,1,7.0,1,7.00,1,6.00,0.136364,9,2.333333,20,2.400000


<h2>product releated feature:-</h2><br>
After research i have found some features that may help us such features are:-<br>




1.   no_of_times_ordered_itemA
2.   no_of_times_reordered_itemA
3.   no_of_user_ordered_itemA
4.   avg_cart_pos_itemA
5.   avg_order_dow_itemA
6.   avg_order_hour_of_day_itemA
7.   avg_days_since_prior_order_itemA
8.   no_of_user_order_aisle_itemA
9.   nb_order_aisle_itemA
10.  nb_reorder_aisle_itemA
11.  mean_add_to_cart_order_of_aisle_itemA
12.  no_of_user_order_department_itemA
13.  nb_order_department_itemA
14.  nb_reorder_department_itemA
15.  mean_add_to_cart_order_of_department_itemA





In [ ]:
#Here we calculated some product releated feture(i.e 1 to 8) for each product_id in prior dataset
nb_order=dict(prior_data.groupby('product_id').size())
nb_reordered=prior_data[prior_data['reordered']==1]
nb_reordered=dict(nb_reordered.groupby('product_id').size())
product_ids=set(prior_data['product_id'])
avg_cart_pos=dict(prior_data.groupby('product_id')['add_to_cart_order'].median().round().apply(int))
prior_order=pd.merge(prior_data,orders,on='order_id',how='inner')
product_order_dow=dict(prior_order.groupby('product_id')['order_dow'].median().round().apply(int))
product_order_hour_of_day=dict(prior_order.groupby('product_id')['order_hour_of_day'].median().round().apply(int))
product_days_since_prior_order=dict(prior_order.groupby('product_id')['days_since_prior_order'].median().round().apply(int))
no_of_user_order=dict(prior_order.groupby(['product_id'])['user_id'].apply(set))
product_df=pd.DataFrame(columns=['product_id','no_of_times_ordered_itemA','no_of_times_reordered_itemA','no_of_user_ordered_itemA','avg_cart_pos_itemA','avg_order_dow_itemA','avg_order_hour_of_day_itemA','avg_days_since_prior_order_itemA'])
for i in tqdm(product_ids):
  if nb_reordered.get(i) is not None:
    product_df=product_df.append({'product_id':i,'no_of_times_ordered_itemA':nb_order.get(i),'no_of_times_reordered_itemA':nb_reordered.get(i),'no_of_user_ordered_itemA':len(no_of_user_order.get(i)),'avg_cart_pos_itemA':avg_cart_pos.get(i),'avg_order_dow_itemA':product_order_dow.get(i),'avg_order_hour_of_day_itemA':product_order_hour_of_day.get(i),'avg_days_since_prior_order_itemA':product_days_since_prior_order.get(i)},ignore_index=True)
  else:
    product_df=product_df.append({'product_id':i,'no_of_times_ordered_itemA':nb_order.get(i),'no_of_times_reordered_itemA':0,'no_of_user_ordered_itemA':len(no_of_user_order.get(i)),'avg_cart_pos_itemA':avg_cart_pos.get(i),'avg_order_dow_itemA':product_order_dow.get(i),'avg_order_hour_of_day_itemA':product_order_hour_of_day.get(i),'avg_days_since_prior_order_itemA':product_days_since_prior_order.get(i)},ignore_index=True)

100%|██████████| 49677/49677 [02:02<00:00, 407.14it/s]


In [ ]:
product_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/product_df.csv', index=False,header=True)

In [ ]:
#Here we calculated some product releated feture with respect to products aisle
data=orders[orders['eval_set']=='prior']
data=pd.merge(data,prior_data,on='order_id',how='inner')
data=pd.merge(data,products,on='product_id',how='inner')
data=pd.merge(data,aisles,on='aisle_id',how='inner')
nb_order_asile=dict(data.groupby('aisle_id').size())
mean_add_to_cart_order_of_aisle=dict(data.groupby('aisle_id')['add_to_cart_order'].mean())
reorder_data=data[data['reordered']==1]
nb_reorder_asile=dict(reorder_data.groupby('aisle_id').size())
no_of_user_order_aisle_itemA=dict(data.groupby(['aisle_id'])['user_id'].apply(set))
aisle_ids=set(data['aisle_id'])
aisles_df=pd.DataFrame(columns=['aisle_id','no_of_user_order_aisle_itemA','nb_order_aisle_itemA','nb_reorder_aisle_itemA','mean_add_to_cart_order_of_aisle_itemA'])
for i in tqdm(aisle_ids):
    aisles_df=aisles_df.append({'aisle_id':i,'no_of_user_order_aisle_itemA':len(no_of_user_order_aisle_itemA.get(i)),'nb_order_aisle_itemA':nb_order_asile.get(i),'nb_reorder_aisle_itemA':nb_reorder_asile.get(i),'mean_add_to_cart_order_of_aisle_itemA':mean_add_to_cart_order_of_aisle.get(i)},ignore_index=True)


100%|██████████| 134/134 [00:00<00:00, 713.63it/s]


In [ ]:
aisles_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/aisles_df.csv', index=False,header=True)

In [ ]:
#Here we calculated some product releated feture with respect to products department
data=orders[orders['eval_set']=='prior']
data=pd.merge(data,prior_data,on='order_id',how='inner')
data=pd.merge(data,products,on='product_id',how='inner')
data=pd.merge(data,departments,on='department_id',how='inner')
nb_order_department=dict(data.groupby('department_id').size())
mean_add_to_cart_order_of_department=dict(data.groupby('department_id')['add_to_cart_order'].mean())
reorder_data=data[data['reordered']==1]
nb_reorder_department=dict(reorder_data.groupby('department_id').size())
no_of_user_order_department_itemA=dict(data.groupby(['aisle_id'])['user_id'].apply(set))
department_ids=set(data['department_id'])
department_df=pd.DataFrame(columns=['department_id','no_of_user_order_department_itemA','nb_order_department_itemA','nb_reorder_department_itemA','mean_add_to_cart_order_of_department_itemA'])
for i in tqdm(department_ids):
    department_df=department_df.append({'department_id':i,"no_of_user_order_department_itemA":len(no_of_user_order_department_itemA.get(i)),'nb_order_department_itemA':nb_order_department.get(i),'nb_reorder_department_itemA':nb_reorder_department.get(i),'mean_add_to_cart_order_of_department_itemA':mean_add_to_cart_order_of_department.get(i)},ignore_index=True)


100%|██████████| 21/21 [00:00<00:00, 194.81it/s]


In [ ]:
department_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/department_df.csv', index=False,header=True)

In [ ]:
#Here we merge product, products aisle and products department into one dataframe product df
product_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/product_df.csv')
aisles_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/aisles_df.csv')
department_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/department_df.csv')
product_df=pd.merge(product_df,products,on='product_id',how='inner')
product_df=pd.merge(product_df,aisles_df,on='aisle_id',how='inner')
product_df=pd.merge(product_df,department_df,on='department_id',how='inner')
product_df=product_df.drop(['product_name', 'aisle_id','department_id'], axis=1)

In [ ]:
product_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/product_df.csv', index=False,header=True)
product_df.head()

In [ ]:
product_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/product_df.csv')
product_df.head()

,product_id,no_of_times_ordered_itemA,no_of_times_reordered_itemA,no_of_user_ordered_itemA,avg_cart_pos_itemA,avg_order_dow_itemA,avg_order_hour_of_day_itemA,avg_days_since_prior_order_itemA,no_of_user_order_aisle_itemA,nb_order_aisle_itemA,nb_reorder_aisle_itemA,mean_add_to_cart_order_of_aisle_itemA,no_of_user_order_department_itemA,nb_order_department_itemA,nb_reorder_department_itemA,mean_add_to_cart_order_of_department_itemA
0,1,1852,1136,716,4,3,13,7,54202.0,234065.0,128431.0,9.253092,85602.0,2887550.0,1657973.0,9.187743
1,78,11,3,8,13,2,14,18,54202.0,234065.0,128431.0,9.253092,85602.0,2887550.0,1657973.0,9.187743
2,102,185,35,150,4,3,12,8,54202.0,234065.0,128431.0,9.253092,85602.0,2887550.0,1657973.0,9.187743
3,172,97,71,26,3,2,12,3,54202.0,234065.0,128431.0,9.253092,85602.0,2887550.0,1657973.0,9.187743
4,285,156,56,100,8,3,13,11,54202.0,234065.0,128431.0,9.253092,85602.0,2887550.0,1657973.0,9.187743


<h3>User related feature:-</h3><br>


1.   no_of_order_by_user
2.   avg_order_size_of_user
3.   no_of_item_ordered_user
4.   no_of_item_reordered_user
5.   avg_overall_reorder_rate_user
6.   avg_day_between_orders_of_user
7.   median_hour_of_day_user_visit
8.   median_order_dow_user
9.   avg_reordered_rate_per_order_user



In [ ]:
#Here we calculated user releated feture for each user_id in prior dataset
user_ids=set(orders[orders['eval_set']=='prior']['user_id'])
nb_orders_user=dict(orders[orders['eval_set']=='prior'].groupby('user_id').size())
order_prior=pd.merge(orders,prior_data,on='order_id',how='inner')
nb_reorder1=dict(order_prior.groupby('user_id').size())
nb_reorder2=order_prior[order_prior['reordered']==1]
nb_reorder2=dict(nb_reorder2.groupby('user_id').size())
agv_day_between_order_user=dict(orders[orders['eval_set']=='prior'].groupby('user_id')['days_since_prior_order'].mean())
avg_time_day_user=dict(orders[orders['eval_set']=='prior'].groupby('user_id')['order_hour_of_day'].median())
median_order_dow_user=dict(orders[orders['eval_set']=='prior'].groupby('user_id')['order_dow'].median())
avg_reordered_rate_per_order_user=pd.merge(prior_data,orders,on='order_id',how='inner')
avg_reordered_rate_per_order_user=avg_reordered_rate_per_order_user.groupby(['user_id','order_id'])['reordered'].mean().reset_index(name='count')
avg_reordered_rate_per_order_user=dict(avg_reordered_rate_per_order_user.groupby('user_id')['count'].mean())

In [ ]:
user_df=pd.DataFrame(columns=['user_id','no_of_order_by_user','avg_order_size_of_user','no_of_item_ordered_user','no_of_item_reordered_user','avg_overall_reorder_rate_user','avg_day_between_orders_of_user','median_hour_of_day_user_visit','median_order_dow_user','avg_reordered_rate_per_order_user'])
for i in tqdm(user_ids):
  if nb_reorder2.get(i) is not None:
    user_df=user_df.append({'user_id':i,'no_of_order_by_user':nb_orders_user.get(i),'avg_order_size_of_user':nb_reorder1.get(i)/nb_orders_user.get(i),'no_of_item_ordered_user':nb_reorder1.get(i),'no_of_item_reordered_user':nb_reorder2.get(i),'avg_overall_reorder_rate_user':float(nb_reorder2.get(i)/nb_reorder1.get(i)),'avg_day_between_orders_of_user':agv_day_between_order_user.get(i),'median_hour_of_day_user_visit':avg_time_day_user.get(i),'median_order_dow_user':median_order_dow_user.get(i),"avg_reordered_rate_per_order_user":avg_reordered_rate_per_order_user.get(i)},ignore_index=True)
  else:
    user_df=user_df.append({'user_id':i,'no_of_order_by_user':nb_orders_user.get(i),'avg_order_size_of_user':nb_reorder1.get(i)/nb_orders_user.get(i),'no_of_item_ordered_user':nb_reorder1.get(i),"no_of_item_reordered_user":0,'avg_overall_reorder_rate_user':0.0,'avg_day_between_orders_of_user':agv_day_between_order_user.get(i),'median_hour_of_day_user_visit':avg_time_day_user.get(i),'median_order_dow_user':median_order_dow_user.get(i),"avg_reordered_rate_per_order_user":avg_reordered_rate_per_order_user.get(i)},ignore_index=True)

100%|██████████| 206209/206209 [09:05<00:00, 378.32it/s]


In [ ]:
user_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/user_df.csv', index=False,header=True)

In [ ]:
#here i merged all user,product,user and product releated feature to one data frame
user_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/user_df.csv')
product_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/product_df.csv')
final_df=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/final_df.csv')
preprocessed=pd.merge(final_df,user_df,on='user_id',how='inner')
preprocessed=pd.merge(preprocessed,product_df,on='product_id',how='inner')
preprocessed.head()

,user_id,product_id,nb_times_userA_order_itemB,avg_pos_cart_userA_itemB,nb_order_userA_itemB_last5,median_day_since_prior_order_userA_itemB,streak_userA_itemB,mean_order_diff_userA_itemB,nb_order_userA_not_ordered_itemB,difference_mean_order_diff_nb_order_not_ordered_userA_itemB,order_ratio_userA_itemB,no_order_aisle_userA_itemB,avg_add_to_cart_order_aisle_userA_itemB,no_order_department_userA_itemB,avg_add_to_cart_order_department_userA_itemB,no_of_order_by_user,avg_order_size_of_user,no_of_item_ordered_user,no_of_item_reordered_user,avg_overall_reorder_rate_user,avg_day_between_orders_of_user,median_hour_of_day_user_visit,median_order_dow_user,avg_reordered_rate_per_order_user,no_of_times_ordered_itemA,no_of_times_reordered_itemA,no_of_user_ordered_itemA,avg_cart_pos_itemA,avg_order_dow_itemA,avg_order_hour_of_day_itemA,avg_days_since_prior_order_itemA,no_of_user_order_aisle_itemA,nb_order_aisle_itemA,nb_reorder_aisle_itemA,mean_add_to_cart_order_of_aisle_itemA,no_of_user_order_department_itemA,nb_order_department_itemA,nb_reorder_department_itemA,mean_add_to_cart_order_of_department_itemA
0,1,196,10,1.400000,5,19.5,10,1.00,0,1.00,1.000000,13,1.769231,13,1.769231,10.0,5.900000,59.0,41.0,0.694915,19.500000,8.5,2.5,0.705833,35791,27791,8000,2,3,12,9,63506.0,357537.0,228406.0,6.453676,14068.0,2690129.0,1757892.0,6.976699
1,15,196,5,2.200000,3,7.0,2,1.75,0,1.75,0.227273,9,2.333333,20,2.400000,22.0,3.272727,72.0,59.0,0.819444,10.801653,11.0,2.0,0.832576,35791,27791,8000,2,3,12,9,63506.0,357537.0,228406.0,6.453676,14068.0,2690129.0,1757892.0,6.976699
2,19,196,3,6.333333,2,6.0,1,2.50,2,0.50,0.333333,6,7.833333,26,12.730769,9.0,22.666667,204.0,71.0,0.348039,9.481481,13.0,5.0,0.414799,35791,27791,8000,2,3,12,9,63506.0,357537.0,228406.0,6.453676,14068.0,2690129.0,1757892.0,6.976699
3,21,196,1,2.000000,0,6.0,1,0.00,23,-23.00,0.030303,6,4.000000,50,3.280000,33.0,6.212121,205.0,103.0,0.502439,9.922865,10.0,1.0,0.554367,35791,27791,8000,2,3,12,9,63506.0,357537.0,228406.0,6.453676,14068.0,2690129.0,1757892.0,6.976699
4,31,196,2,15.500000,1,7.5,1,7.00,3,4.00,0.100000,27,15.222222,76,11.407895,20.0,14.950000,299.0,109.0,0.364548,4.927500,11.0,2.5,0.281645,35791,27791,8000,2,3,12,9,63506.0,357537.0,228406.0,6.453676,14068.0,2690129.0,1757892.0,6.976699


In [ ]:
preprocessed.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/preprocessed.csv', index=False,header=True)

In [ ]:
preprocessed_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/preprocessed.csv')

In [ ]:
def get_data(flag='train'):
  '''this function will get the current features releted to next order and all history features of that user if the user belongs to train data then get the output variable i.e reordered'''
  preprocessed_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/data/preprocessed.csv')
  orders = pd.read_csv('orders.csv')
  temp=orders[orders['eval_set']==flag][['order_id','user_id','order_dow','order_hour_of_day','days_since_prior_order']]  
  preprocessed_data=pd.merge(preprocessed_data,temp,on='user_id',how='inner')
  if flag=='train':
    train_data = pd.read_csv('order_products__train.csv')
    temp2=pd.merge(orders,train_data,on='order_id',how='inner')[['user_id','product_id','reordered']]
    preprocessed_data=pd.merge(preprocessed_data,temp2,on=['user_id','product_id'],how='left')
    for i in range(len(preprocessed_data)):
      if str(preprocessed_data['reordered'][i])=='nan':
        preprocessed_data['reordered'].values[i]=0
  preprocessed_data=preprocessed_data.drop(['order_id'], axis=1)
  return preprocessed_data

In [ ]:
import pandas as pd
train_data=get_data(flag='train')
test_data=get_data(flag='test')


In [ ]:
train_data.shape,test_data.shape

((8474661, 43), (4833292, 42))